In [287]:
import numpy as np
import pandas as pd
import json
from pprint import pprint
from splinter import Browser 
from bs4 import BeautifulSoup as bs
import time
import collections
import json
import csv

In [288]:
data = pd.read_csv("NYCinspection.csv") 
data.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,50000598,UNCLE TED'S,MANHATTAN,163,BLEECKER STREET,10012.0,2127771395,Chinese,10/3/2016,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,46.0,NaN,NaN,2/6/2019,Cycle Inspection / Initial Inspection
1,50000786,DUNKIN' DONUTS,BROOKLYN,873,FLATBUSH AVENUE,11226.0,7189400080,Donuts,12/14/2017,Violations were cited in the following area(s).,10H,Proper sanitization not provided for utensil w...,Not Critical,10.0,NaN,NaN,2/6/2019,Cycle Inspection / Initial Inspection
2,41703747,M2N BUFFET,QUEENS,4012,82 STREET,11373.0,7188032488,American,1/27/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,21.0,NaN,NaN,2/6/2019,Cycle Inspection / Initial Inspection
3,40394258,TOMMASO RESTAURANT,BROOKLYN,1464,86 STREET,11228.0,7182369883,Italian,10/24/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,27.0,B,10/24/2017,2/6/2019,Cycle Inspection / Re-inspection
4,50059515,STICKY'S FINGER JOINT,MANHATTAN,21,MAIDEN LN,10038.0,9177551770,Chicken,3/8/2018,Violations were cited in the following area(s).,06A,Personal cleanliness inadequate. Outer garment...,Critical,5.0,A,3/8/2018,2/6/2019,Cycle Inspection / Initial Inspection


In [289]:
data = data.drop(['CAMIS','BUILDING','PHONE','INSPECTION DATE','ACTION','INSPECTION TYPE'], axis=1)
data.head(5)


,DBA,BORO,STREET,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE
0,UNCLE TED'S,MANHATTAN,BLEECKER STREET,10012.0,Chinese,04N,Filth flies or food/refuse/sewage-associated (...,Critical,46.0,NaN,NaN,2/6/2019
1,DUNKIN' DONUTS,BROOKLYN,FLATBUSH AVENUE,11226.0,Donuts,10H,Proper sanitization not provided for utensil w...,Not Critical,10.0,NaN,NaN,2/6/2019
2,M2N BUFFET,QUEENS,82 STREET,11373.0,American,10F,Non-food contact surface improperly constructe...,Not Critical,21.0,NaN,NaN,2/6/2019
3,TOMMASO RESTAURANT,BROOKLYN,86 STREET,11228.0,Italian,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,27.0,B,10/24/2017,2/6/2019
4,STICKY'S FINGER JOINT,MANHATTAN,MAIDEN LN,10038.0,Chicken,06A,Personal cleanliness inadequate. Outer garment...,Critical,5.0,A,3/8/2018,2/6/2019


In [290]:
data = data.dropna()
data_gb = data.groupby(['ZIPCODE','BORO'])['VIOLATION CODE'].count()
data_gb.head(10)

ZIPCODE  BORO     
10000.0  MANHATTAN      15
10001.0  MANHATTAN    3495
10002.0  MANHATTAN    3512
10003.0  MANHATTAN    4762
10004.0  MANHATTAN    1015
10005.0  MANHATTAN     497
10006.0  MANHATTAN     325
10007.0  MANHATTAN    1044
10009.0  MANHATTAN    2152
10010.0  MANHATTAN    1804
Name: VIOLATION CODE, dtype: int64

In [291]:
data_df = pd.DataFrame(data_gb)
data_df.reset_index
data_df.to_csv('nycrest.csv', sep=',', encoding='utf-8')

In [292]:
nyc_rest = pd.read_csv("nycrest.csv") 
nyc_rest.head(20)

,ZIPCODE,BORO,VIOLATION CODE
0,10000.0,MANHATTAN,15
1,10001.0,MANHATTAN,3495
2,10002.0,MANHATTAN,3512
3,10003.0,MANHATTAN,4762
4,10004.0,MANHATTAN,1015
5,10005.0,MANHATTAN,497
6,10006.0,MANHATTAN,325
7,10007.0,MANHATTAN,1044
8,10009.0,MANHATTAN,2152
9,10010.0,MANHATTAN,1804


In [293]:
nyc_rest = nyc_rest.rename(index=str, columns={"VIOLATION CODE": "VIOLATIONS"})
nyc_rest.head()

,ZIPCODE,BORO,VIOLATIONS
0,10000.0,MANHATTAN,15
1,10001.0,MANHATTAN,3495
2,10002.0,MANHATTAN,3512
3,10003.0,MANHATTAN,4762
4,10004.0,MANHATTAN,1015


In [294]:
nyc_rest["ZIPCODE"] = nyc_rest["ZIPCODE"].astype('int64')
nyc_rest.head(20)

,ZIPCODE,BORO,VIOLATIONS
0,10000,MANHATTAN,15
1,10001,MANHATTAN,3495
2,10002,MANHATTAN,3512
3,10003,MANHATTAN,4762
4,10004,MANHATTAN,1015
5,10005,MANHATTAN,497
6,10006,MANHATTAN,325
7,10007,MANHATTAN,1044
8,10009,MANHATTAN,2152
9,10010,MANHATTAN,1804


In [295]:
data2 = pd.read_csv("food_poisoning.csv")
data2.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,41768057,02/22/2019 03:40:21 PM,NaN,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,11211.0,172 GRAND STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.714403,-73.961067,"(40.71440253567804, -73.96106715130767)"
1,41767453,02/22/2019 01:15:00 PM,NaN,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,10457.0,470 EAST TREMONT AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.846990,-73.897911,"(40.84698951872327, -73.89791065009413)"
2,41765289,02/22/2019 07:55:49 PM,NaN,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,10029.0,1407 MADISON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.788157,-73.953469,"(40.788156566910075, -73.95346933449004)"
3,41764391,02/22/2019 12:02:54 PM,NaN,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Other (Explain Below),11435.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.690018,-73.803232,"(40.69001834357505, -73.80323242165294)"
4,41764235,02/22/2019 12:00:00 AM,02/22/2019 12:53:11 PM,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,11102.0,28-46 31 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.767537,-73.920874,"(40.76753679786473, -73.92087361582338)"


In [296]:
filtered = data2.filter(['Complaint Type','Location Type','Incident Zip','Borough'],axis=1)
filtered

,Complaint Type,Location Type,Incident Zip,Borough
0,Food Poisoning,Restaurant/Bar/Deli/Bakery,11211.0,BROOKLYN
1,Food Poisoning,Restaurant/Bar/Deli/Bakery,10457.0,BRONX
2,Food Poisoning,Restaurant/Bar/Deli/Bakery,10029.0,MANHATTAN
3,Food Poisoning,Other (Explain Below),11435.0,QUEENS
4,Food Poisoning,Restaurant/Bar/Deli/Bakery,11102.0,QUEENS
5,Food Poisoning,Restaurant/Bar/Deli/Bakery,10013.0,MANHATTAN
6,Food Poisoning,Restaurant/Bar/Deli/Bakery,10019.0,MANHATTAN
7,Food Poisoning,Restaurant/Bar/Deli/Bakery,10016.0,MANHATTAN
8,Food Poisoning,Restaurant/Bar/Deli/Bakery,10075.0,MANHATTAN
9,Food Poisoning,Restaurant/Bar/Deli/Bakery,11375.0,QUEENS


In [297]:
filtered = filtered.dropna()
filtered_gb = filtered.groupby(['Incident Zip','Borough'])['Complaint Type'].count()
filtered_gb.head(20)

Incident Zip  Borough    
0.0           Unspecified      1
100.0         Unspecified      1
1106.0        QUEENS           1
10000.0       MANHATTAN        8
              Unspecified      1
10001.0       MANHATTAN      512
10002.0       MANHATTAN      354
10003.0       MANHATTAN      721
              Unspecified      1
10004.0       MANHATTAN      115
10005.0       MANHATTAN       55
10006.0       MANHATTAN       38
10007.0       MANHATTAN      185
              Unspecified      1
10009.0       MANHATTAN      251
10010.0       MANHATTAN      260
10011.0       MANHATTAN      532
10012.0       MANHATTAN      346
10013.0       MANHATTAN      445
10014.0       MANHATTAN      308
Name: Complaint Type, dtype: int64

In [298]:
filtered_df = pd.DataFrame(filtered_gb)
data_df.reset_index
filtered_df.to_csv('nycfp.csv', sep=',', encoding='utf-8')

In [299]:
nyc_fp = pd.read_csv("nycfp.csv") 
nyc_fp = nyc_fp[nyc_fp.Borough != 'Unspecified']
nyc_fp

,Incident Zip,Borough,Complaint Type
2,1106.0,QUEENS,1
3,10000.0,MANHATTAN,8
5,10001.0,MANHATTAN,512
6,10002.0,MANHATTAN,354
7,10003.0,MANHATTAN,721
9,10004.0,MANHATTAN,115
10,10005.0,MANHATTAN,55
11,10006.0,MANHATTAN,38
12,10007.0,MANHATTAN,185
14,10009.0,MANHATTAN,251


In [300]:
nyc_fp = nyc_fp.rename(index=str, columns={"Borough": "BORO", "Incident Zip": "ZIPCODE2",'Complaint Type':'FP_Count'})
nyc_fp["ZIPCODE2"] = nyc_fp["ZIPCODE2"].astype('int64')
nyc_fp.head(20)

,ZIPCODE2,BORO,FP_Count
2,1106,QUEENS,1
3,10000,MANHATTAN,8
5,10001,MANHATTAN,512
6,10002,MANHATTAN,354
7,10003,MANHATTAN,721
9,10004,MANHATTAN,115
10,10005,MANHATTAN,55
11,10006,MANHATTAN,38
12,10007,MANHATTAN,185
14,10009,MANHATTAN,251


In [301]:
nyc_rest = nyc_rest.drop(['BORO'],axis=1)
nyc_rest.head()

,ZIPCODE,VIOLATIONS
0,10000,15
1,10001,3495
2,10002,3512
3,10003,4762
4,10004,1015


In [302]:
nyc_fp.head()

,ZIPCODE2,BORO,FP_Count
2,1106,QUEENS,1
3,10000,MANHATTAN,8
5,10001,MANHATTAN,512
6,10002,MANHATTAN,354
7,10003,MANHATTAN,721


In [318]:
joined = nyc_fp.set_index('ZIPCODE2').join(nyc_rest.set_index('ZIPCODE'))
joined = joined.fillna(0)
joined.index.name = "ZIPCODE"
joined=joined.reset_index()
joined = joined.rename(index=str, columns={"FP_Count": "FOOD POISONING CNT", "VIOLATIONS": "FOOD INSPECTION VIOLATIONS"})
joined["FOOD INSPECTION VIOLATIONS"] = joined["FOOD INSPECTION VIOLATIONS"].astype('int64')
joined.head()

,ZIPCODE,BORO,FOOD POISONING CNT,FOOD INSPECTION VIOLATIONS
0,1106,QUEENS,1,0
1,10000,MANHATTAN,8,15
2,10001,MANHATTAN,512,3495
3,10002,MANHATTAN,354,3512
4,10003,MANHATTAN,721,4762


In [319]:
joined.to_csv('NYC FP vs FIV.csv', sep=',', encoding='utf-8')